In [0]:
# примонтировать данные с гугл-диска
# пи запуске с локальной машины это не нужно

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import gensim
import numpy as np
from nltk.util import skipgrams
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score, cross_val_predict

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import SGDClassifier 
from sklearn.linear_model import LassoCV
from sklearn.naive_bayes import BernoulliNB, BaseNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from nltk.corpus import stopwords

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import nltk.data
nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/roman/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

In [3]:
# dpred= pd.read_csv("/content/gdrive/My Drive/NLP/works/wrk01_sentiment/product-reviews-sentiment-analysis-light/products_sentiment_test.tsv",sep="\t")

# dtrain = pd.read_csv(
#     "/content/gdrive/My Drive/NLP/works/wrk01_sentiment/product-reviews-sentiment-analysis-light/products_sentiment_train.tsv", 
#     sep="\t",
#     header=None,
#     names=["text", "response"]
# )

dpred= pd.read_csv("../product-reviews-sentiment-analysis-light/products_sentiment_test.tsv",sep="\t")

dtrain = pd.read_csv(
    "../product-reviews-sentiment-analysis-light/products_sentiment_train.tsv", 
    sep="\t",
    header=None,
    names=["text", "response"]
)

cv = RepeatedKFold(n_repeats=4, n_splits=5)

# Simple Vectorizing

In [0]:
pipe = Pipeline(
    [
        ('vec', CountVectorizer(stop_words="english")),
        ('clf', LogisticRegression())
    ]
)

scores = cross_val_score(pipe, dtrain.text, dtrain.response, cv=cv)
print(f"avg={scores.mean()} std={scores.std()}")

avg=0.7541249999999999 std=0.015717724867168267


In [20]:
pipe = Pipeline(
    [
        ('vec', TfidfVectorizer(ngram_range=(1, 2))), #avg=0.7444 std=0.012
#         ('vec', TfidfVectorizer(stop_words="english")), #avg=0.7179 std=0.018
        ('clf', LogisticRegressionCV(max_iter=1000, cv=RepeatedKFold(n_repeats=2, n_splits=10)))
    ]
)

scores = cross_val_score(pipe, dtrain.text, dtrain.response, cv=RepeatedKFold(n_repeats=10, n_splits=2))
print(f"avg={scores.mean()} std={scores.std()}")
# avg=0.7646000000000001 std=0.010016985574512931
# avg=0.7659 std=0.013018832512940639

avg=0.76485 std=0.012527070687115975


In [0]:
####### for feature analysis ######

# dtrain['pred'] = pred[:, 1]
# dtrain['aerr'] = abs(dtrain.pred - dtrain.response)

# dtrain.sort_values(by='aerr', ascending=False)

# pipe.fit(dtrain.text, dtrain.response)

# vec = pipe.named_steps['vec']
# res = vec.transform([dtrain.text[15]]).todense()
# list(filter(lambda x: x[1]!=0, zip(vec.get_feature_names(),res.tolist()[0])))

# def get_wei(word):
#     word_idx = pipe.named_steps['vec'].vocabulary_.get(word, None)
#     return pipe.named_steps['clf'].coef_[0][word_idx] if word_idx is not None else None

# txt = dtrain.text[488].split()
# vec = pipe.named_steps['vec'].transform([dtrain.text[15]]).todense()
# dict(zip(txt,list(map(get_wei, txt))))


In [0]:
pipe = Pipeline(
    [
        ('vec', TfidfVectorizer()),#stop_words="english")),
        ('clf', BernoulliNB())
    ]
)

scores = cross_val_score(pipe, dtrain.text, dtrain.response, cv=RepeatedKFold(n_repeats=4, n_splits=10))
print(f"avg={scores.mean()} std={scores.std()}")

avg=0.75 std=0.030145480589965735


# PREDICT

## vesrion 1

In [0]:
# pipe = Pipeline(
#     [
#         ('vec', TfidfVectorizer(stop_words="english")),
#         ('clf', LogisticRegression())
#     ]
# )

# pipe.fit(dtrain.text, dtrain.response)
# dresult = pd.DataFrame({
#     "Id": dpred.Id,
#     "y": pipe.predict(dpred.text)
# })
# dresult.to_csv("tfidf_lr.csv", index=None)

## version 2

In [12]:
pipe = Pipeline(
    [
        ('vec', TfidfVectorizer(ngram_range=(1, 2))),
        ('clf', LogisticRegressionCV(max_iter=500))
    ]
)

pipe.fit(dtrain.text, dtrain.response)
# dresult = pd.DataFrame({
#     "Id": dpred.Id,
#     "y": pipe.predict(dpred.text)
# })
# dresult.to_csv("tfidf_lr_2.csv", index=None)

# 0.825003 on LB

Pipeline(memory=None,
     steps=[('vec', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...    random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0))])

In [13]:
import pickle

with open('vec.pkl', 'wb') as f:
    pickle.dump(pipe.named_steps['vec'], f)

In [14]:
with open('clf.pkl', 'wb') as f:
    pickle.dump(pipe.named_steps['clf'], f)

In [15]:
with open('vec.pkl', 'rb') as f:
    data_new = pickle.load(f)
    

In [16]:
data_new.transform(["cat"])

<1x22786 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

## version 3

In [0]:
# pipe = Pipeline(
#     [
#         ('vec', TfidfVectorizer(ngram_range=(1, 2))),
#         ('clf', LogisticRegressionCV(max_iter=500, cv=RepeatedKFold(n_repeats=2, n_splits=10)))
#     ]
# )

# pipe.fit(dtrain.text, dtrain.response)
# dresult = pd.DataFrame({
#     "Id": dpred.Id,
#     "y": pipe.predict(dpred.text)
# })
# dresult.to_csv("tfidf_lr_3.csv", index=None)

# # 0.81750 on LB